In [1]:
#Multiple Processes Sharing a File
#https://stackoverflow.com/questions/13446445/python-multiprocessing-safely-writing-to-a-file
#Have a Separate Listener Process for writing to file

In [2]:
import multiprocessing as mp

In [3]:
def writer(seed,log_q):
    log_q.put(seed)

def log_writer(log_q):
    with open("test.txt",'w') as f:
        while 1:
            m = log_q.get()
            if m == 'kill':
                break
            f.write(str(m) + '\n')
            f.flush()


In [4]:
#Prepare Manager & Queue
manager = mp.Manager()
log_q = manager.Queue()

In [5]:
#Multiprocessing Pool
#mp.cpu_count()+2 is rule of thumb to keep cpu saturated
pool = mp.Pool(mp.cpu_count() + 2)

In [6]:
#Start Separate process for file io
watcher = pool.apply_async(log_writer, (log_q,))

In [7]:
#Start Processes
seeds=range(10,160,10)
processes=[]
for seed in seeds:
    job = pool.apply_async(writer, (seed, log_q))
    processes.append(job)

In [8]:
for p in processes:
    p.get()

In [9]:
#Kill Listener Process
log_q.put('kill')
pool.close()
pool.join()